In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
from datetime import datetime
import json
from pathlib import Path

# --- Setup ---
mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient()

# --- INPUTS ---
model_name = "milk-price-predictor-rf"

# Get latest version in "Staging"
versions = client.get_latest_versions(model_name, stages=["Staging"])
if not versions:
    raise ValueError(f"No model found in Staging stage for '{model_name}'")

model_version = versions[0].version
run_id = versions[0].run_id
print(f"📌 Latest '{model_name}' in Staging: version {model_version}, run_id {run_id}")

# Get model path from model registry (not run)
registered_model = client.get_model_version(name=model_name, version=model_version)
artifact_uri = registered_model.source  # this points to s3://.../models/m-xxx/artifacts

print(f"📦 artifact_uri: {artifact_uri}")

# Get RMSE from run (metric must have been logged as 'final_rmse')
run = client.get_run(run_id)
rmse = float(run.data.metrics.get("final_rmse", -1))
print(f"📈 final_rmse: {rmse}")

# --- Build promotion record ---
promotion_record = {
    "model_name": model_name,
    "version": str(model_version),
    "run_id": run_id,
    "artifact_uri": artifact_uri,
    "rmse": rmse,
    "promoted_stage": "Staging",
    "promotion_time": datetime.utcnow().isoformat()
}

# --- Save locally for testing ---
Path("deployment/promoted").mkdir(parents=True, exist_ok=True)

with open("deployment/promoted/daily_model.json", "w") as f:
    json.dump(promotion_record, f, indent=2)

print("✅ daily_model.json saved locally:")
print(json.dumps(promotion_record, indent=2))


📌 Latest 'milk-price-predictor-rf' in Staging: version 2, run_id 94127e382abb4a53a17632764c844261
📦 artifact_uri: models:/m-7e67bea8560d44c4b4e38bad4030982d
📈 final_rmse: 0.21774853869800637
✅ daily_model.json saved locally:
{
  "model_name": "milk-price-predictor-rf",
  "version": "2",
  "run_id": "94127e382abb4a53a17632764c844261",
  "artifact_uri": "models:/m-7e67bea8560d44c4b4e38bad4030982d",
  "rmse": 0.21774853869800637,
  "promoted_stage": "Staging",
  "promotion_time": "2025-08-01T04:57:35.411224"
}


/tmp/ipykernel_38224/1374647366.py:15: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(model_name, stages=["Staging"])
/tmp/ipykernel_38224/1374647366.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "promotion_time": datetime.utcnow().isoformat()


In [3]:
registered_model.source

'models:/m-7e67bea8560d44c4b4e38bad4030982d'

In [4]:
run

<Run: data=<RunData: metrics={'final_rmse': 0.21774853869800637}, params={}, tags={'mlflow.runName': 'rf-milk-predictor-2025-07',
 'mlflow.source.git.commit': 'b55658938e19e2201907b55339c86c96f2e41540',
 'mlflow.source.name': 'orchestration/flows/master_daily_flow.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'maxkaizo',
 'model_type': 'random_forest'}>, info=<RunInfo: artifact_uri='s3://mlflow-models-milk-price-dev/2/94127e382abb4a53a17632764c844261/artifacts', end_time=1754018234207, experiment_id='2', lifecycle_stage='active', run_id='94127e382abb4a53a17632764c844261', run_name='rf-milk-predictor-2025-07', start_time=1754016831438, status='FINISHED', user_id='maxkaizo'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=<RunOutputs: model_outputs=[<LoggedModelOutput: model_id='m-7e67bea8560d44c4b4e38bad4030982d', step=0>]>>